In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
from time import sleep
import json


def save_json(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f)

#### Scrapping Data

In [2]:
url = 'https://edtools.cc/pve'
params = {
    's': 'Achenar', 
    'md': '250', 
    'sc': '2'
}
# We need the headers here in this website.
# By doing so, the website will think that someone is actually visiting the site using a web browser.
# Ref.: https://stackoverflow.com/questions/61968521/python-web-scraping-request-errormod-security
# Check headers params: https://httpbin.org/headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.71",
}

# Request
r = requests.get(url, params=params, headers=headers)

# Soup
soup = bs(r.content)

In [3]:
table_data = soup.find('table', id='sys_tbl')

In [4]:
with open('systems_scrape.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    table_head = []
    table_body = []
    for index, tr in enumerate(table_data.find_all('tr')):
        if index == 0:
            for th in tr.find_all('th'):
                table_head.append(th.text.strip())
            table_head.append('URL')
            csv_writer.writerow(table_head)
        elif index > 0:
            for i, td in enumerate(tr.find_all('td')):
                table_body.append(td.text.strip())
                if i == 9:
                    unique_factions_url = url + td.find_all('a')[-1]['href']
            table_body.append(unique_factions_url)
            csv_writer.writerow(table_body)
            table_body.clear()

#### DataFrame

In [5]:
df = pd.read_csv('systems_scrape.csv', thousands=',')

In [6]:
df.dtypes

Distance          float64
Source System      object
L-Pad             float64
M-pad             float64
P-pad             float64
Fed               float64
Imp               float64
All               float64
Ind               float64
Target/Sources     object
RES/rings          object
URL                object
dtype: object

#### Filtering

In [7]:
dfa = df.loc[(df['L-Pad'] < 2000)].reset_index(drop=True)
dfa

,Distance,Source System,L-Pad,M-pad,P-pad,Fed,Imp,All,Ind,Target/Sources,RES/rings,URL
0,37.22,LHS 1071,793.0,3124.0,1597.0,NaN,5.0,NaN,2.0,Mundjiga / 2,No rings,https://edtools.cc/pve?s=Mundjiga&a=rt
1,44.68,Themiscrya,616.0,NaN,NaN,NaN,3.0,NaN,4.0,CD-41 568 / 2,3 rings,https://edtools.cc/pve?s=CD-41+568&a=rt
2,49.11,CD-61 6801,1036.0,NaN,404.0,NaN,4.0,NaN,3.0,Findja / 2,No rings,https://edtools.cc/pve?s=Findja&a=rt
3,49.12,Elysia,521.0,NaN,483.0,NaN,4.0,NaN,3.0,Nandigamara / 2,1 ring,https://edtools.cc/pve?s=Nandigamara&a=rt
4,51.34,Phiagre,27.0,NaN,26.0,NaN,3.0,NaN,4.0,Findja / 2,No rings,https://edtools.cc/pve?s=Findja&a=rt
...,...,...,...,...,...,...,...,...,...,...,...,...
212,248.17,Cuachini,6.0,11.0,6.0,NaN,NaN,NaN,7.0,Weleutaheim / 2,1 ring,https://edtools.cc/pve?s=Weleutaheim&a=rt
213,248.87,Palliyan,598.0,968.0,969.0,2.0,NaN,NaN,5.0,HIP 55288 / 2,5 rings,https://edtools.cc/pve?s=HIP+55288&a=rt
214,248.90,Jura Laima,356.0,NaN,NaN,1.0,NaN,NaN,5.0,HIP 109310 / 2,3 rings,https://edtools.cc/pve?s=HIP+109310&a=rt
215,249.28,Hou Hsien,632.0,NaN,317.0,NaN,NaN,4.0,3.0,GCRV 7300 / 2,4 rings,https://edtools.cc/pve?s=GCRV+7300&a=rt


#### DataFrame with only duplicated Target/Sources

In [8]:
dfb = dfa['Target/Sources'].value_counts().reset_index()
dfb.loc[dfb['Target/Sources'] > 1]

,index,Target/Sources
0,Na Chem / 3,3
1,Wader / 2,2
2,HIP 108701 / 3,2
3,Khodia / 2,2
4,Wolf 186 / 2,2
5,Hun Nighu / 2,2
6,Anana / 3,2
7,Sairre / 2,2
8,Oxlahatta / 2,2
9,Skappa / 2,2


In [9]:
dfc = dfa[dfa.duplicated(subset=['Target/Sources'], keep=False)].copy()
dfc.reset_index(drop=True)

,Distance,Source System,L-Pad,M-pad,P-pad,Fed,Imp,All,Ind,Target/Sources,RES/rings,URL
0,49.11,CD-61 6801,1036.0,NaN,404.0,NaN,4.0,NaN,3.0,Findja / 2,No rings,https://edtools.cc/pve?s=Findja&a=rt
1,51.34,Phiagre,27.0,NaN,26.0,NaN,3.0,NaN,4.0,Findja / 2,No rings,https://edtools.cc/pve?s=Findja&a=rt
2,52.64,HIP 12716,547.0,1699.0,4267.0,NaN,4.0,NaN,3.0,HIP 13841 / 2,4 rings,https://edtools.cc/pve?s=HIP+13841&a=rt
3,60.44,Nayanezgani,350.0,NaN,2343.0,NaN,4.0,NaN,3.0,Slatas / 2,1 ring,https://edtools.cc/pve?s=Slatas&a=rt
4,62.22,LTT 982,1941.0,NaN,1941.0,NaN,5.0,NaN,2.0,Chimechilo / 2,No rings,https://edtools.cc/pve?s=Chimechilo&a=rt
...,...,...,...,...,...,...,...,...,...,...,...,...
66,231.68,HIP 70681,267.0,508.0,NaN,1.0,NaN,NaN,6.0,Obates / 2,1 ring,https://edtools.cc/pve?s=Obates&a=rt
67,238.81,Wolf 631,1633.0,4296.0,1628.0,2.0,NaN,NaN,5.0,HIP 82042 / 2,5 rings,https://edtools.cc/pve?s=HIP+82042&a=rt
68,241.25,LTT 13904,78.0,NaN,141.0,NaN,NaN,4.0,3.0,BD+37 2416 / 2,No rings,https://edtools.cc/pve?s=BD%2B37+2416&a=rt
69,242.89,Ross 809,196.0,NaN,NaN,3.0,NaN,1.0,3.0,HIP 82042 / 2,5 rings,https://edtools.cc/pve?s=HIP+82042&a=rt


In [15]:
# Uncomment cell just to visualize the data

for target_source, frame in dfc.groupby('Target/Sources'):
    print(f"First 2 entries for {target_source!r}")
    print("------------------------")
    print(frame, end="\n\n")

First 2 entries for 'Amitae / 2'
------------------------
     Distance Source System   L-Pad  M-pad  P-pad  Fed  Imp  All  Ind  \
97     168.28       Lhanayi  1446.0    NaN  660.0  NaN  NaN  NaN  6.0   
107    173.87         Nugua    59.0   15.0   15.0  1.0  NaN  NaN  6.0   

    Target/Sources RES/rings                                   URL  
97      Amitae / 2  No rings  https://edtools.cc/pve?s=Amitae&a=rt  
107     Amitae / 2  No rings  https://edtools.cc/pve?s=Amitae&a=rt  

First 2 entries for 'Anana / 3'
------------------------
    Distance Source System  L-Pad  M-pad  P-pad  Fed  Imp  All  Ind  \
92    164.51        Ahayan  690.0  915.0    NaN  1.0  NaN  NaN  6.0   
95    166.98     HIP 90112  237.0  557.0  557.0  1.0  NaN  NaN  6.0   

   Target/Sources RES/rings                                  URL  
92      Anana / 3    1 ring  https://edtools.cc/pve?s=Anana&a=rt  
95      Anana / 3    1 ring  https://edtools.cc/pve?s=Anana&a=rt  

First 2 entries for 'Atun / 2'
----------

In [11]:
def check_faction_state(state):
    bag_of_words = ['war', 'election', 'civil war']
    if any(word in state.lower() for word in bag_of_words):
        return True
    

unique_factions_data = {}
count = 0
for target_source, frame in dfc.groupby('Target/Sources'):
    
    count += 1
    new_r = requests.get(frame['URL'].values[0], headers=headers)
    new_soup = bs(new_r.content)
    
    table_body = {}
    keep_going = True
    for index, tr in enumerate(new_soup.body.table.find_all('tr')):
        if not keep_going:
            break
        elif index > 0:
            rows = []
            for i, td in enumerate(tr.find_all('td')):
                if i == 0:
                    key = td.text.strip()
                else:
                    value = td.text.strip()
                    rows.append(value)
                    # Checking faction state
#                     if len(rows) == 4:
#                         if check_faction_state(rows[3]):
#                             table_body.clear()
#                             keep_going = False
#                             break
                    table_body[key] = rows
                    
    if table_body:           
        unique_factions_data[frame['Target/Sources'].values[0]] = table_body.copy()
    if count % 10 == 0:
        print(f'{count} iterations.')
    sleep(new_r.elapsed.total_seconds())

10 iterations.
20 iterations.
30 iterations.


In [12]:
print(json.dumps(unique_factions_data, indent=2))

{
  "Amitae / 2": {
    "1": [
      "The Ken'Tarii Mandate",
      "Independent",
      "Theocracy",
      "Investment",
      "Lhanayi, Nugua"
    ],
    "2": [
      "United Lhanayi Defence Force",
      "Independent",
      "Dictatorship",
      "Civil war",
      "Lhanayi"
    ],
    "3": [
      "Lhanayi General PLC",
      "Independent",
      "Corporate",
      "Civil war",
      "Lhanayi"
    ],
    "4": [
      "Lhanayi Confederacy",
      "Independent",
      "Confederacy",
      "Infrastructure Failure",
      "Lhanayi"
    ],
    "5": [
      "United Wang Dana Free",
      "Federation",
      "Democracy",
      "None",
      "Nugua"
    ],
    "6": [
      "United Nugua Freedom Party",
      "Independent",
      "Dictatorship",
      "None",
      "Nugua"
    ],
    "7": [
      "Nugua Revered Cult",
      "Independent",
      "Theocracy",
      "None",
      "Nugua"
    ],
    "8": [
      "Nugua Crimson Dynamic Limited",
      "Independent",
      "Corporate",
      "Non

#### Saving in Json

In [13]:
save_json('unique_factions.json', unique_factions_data)

#### Loading Json

In [14]:
with open('unique_factions.json') as f:
    parsed = json.load(f)
    data = json.dumps(parsed, indent=2)
    print(data)

{
  "Amitae / 2": {
    "1": [
      "The Ken'Tarii Mandate",
      "Independent",
      "Theocracy",
      "Investment",
      "Lhanayi, Nugua"
    ],
    "2": [
      "United Lhanayi Defence Force",
      "Independent",
      "Dictatorship",
      "Civil war",
      "Lhanayi"
    ],
    "3": [
      "Lhanayi General PLC",
      "Independent",
      "Corporate",
      "Civil war",
      "Lhanayi"
    ],
    "4": [
      "Lhanayi Confederacy",
      "Independent",
      "Confederacy",
      "Infrastructure Failure",
      "Lhanayi"
    ],
    "5": [
      "United Wang Dana Free",
      "Federation",
      "Democracy",
      "None",
      "Nugua"
    ],
    "6": [
      "United Nugua Freedom Party",
      "Independent",
      "Dictatorship",
      "None",
      "Nugua"
    ],
    "7": [
      "Nugua Revered Cult",
      "Independent",
      "Theocracy",
      "None",
      "Nugua"
    ],
    "8": [
      "Nugua Crimson Dynamic Limited",
      "Independent",
      "Corporate",
      "Non

#### DataFrame Analysis

In [15]:
df_parsed = pd.DataFrame.from_dict(parsed, orient='index')
df_parsed

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Amitae / 2,"[The Ken'Tarii Mandate, Independent, Theocracy...","[United Lhanayi Defence Force, Independent, Di...","[Lhanayi General PLC, Independent, Corporate, ...","[Lhanayi Confederacy, Independent, Confederacy...","[United Wang Dana Free, Federation, Democracy,...","[United Nugua Freedom Party, Independent, Dict...","[Nugua Revered Cult, Independent, Theocracy, N...","[Nugua Crimson Dynamic Limited, Independent, C...","[Order of Nugua, Independent, Dictatorship, No...","[Nugua Jet Advanced Partners, Independent, Cor...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
Anana / 3,"[Labour Union of Ahayan, Independent, Communis...","[Ahayan Gold Creative Co, Federation, Corporat...","[Liberals of Ahayan, Independent, Democracy, N...","[Workers of Dimocorna Union, Independent, Comm...","[Ahayan Defence Party, Independent, Dictatorsh...","[HR 7169 Union Party, Independent, Communism, ...","[HIP 90112 Liberty Party, Independent, Dictato...","[New HIP 90112 for Equality, Federation, Democ...","[HIP 90112 Jet Central Corp., Independent, Cor...","[Natural HIP 90112 Party, Independent, Dictato...","[Puneith Values Party, Federation, Democracy, ...","[Puneith Organisation, Independent, Corporate,...","[United Tagii League, Independent, Confederacy...","[Crimson Armada, Independent, Confederacy, Non...",NaN,NaN,NaN
Atun / 2,"[Brazilian League of Pilots, Independent, Conf...","[League of Cava Law Party, Independent, Dictat...","[Cava PLC, Independent, Corporate, None, Cava]","[Movement for Cava Unionists, Independent, Com...","[Beatis Collective of Marlinists, Independent,...","[Liberty Party of Na Chac Og, Independent, Dic...","[Na Chac Og Freedom Party, Independent, Dictat...","[Na Chac Og Incorporated, Independent, Corpora...","[People's Na Chac Og Democrats, Independent, D...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BD+37 2416 / 2,"[Sol's Salutis Aeternum, Alliance, Cooperative...","[Sirius Corporation, Independent, Corporate, N...","[Di Jian Alliance Mandate, Alliance, Patronage...","[Di Jian Crimson Life Corp., Independent, Corp...","[Di Jian Dynamic Systems, Independent, Corpora...","[United Di Jian Progressive Party, Alliance, D...","[New Di Jian Constitution Party, Independent, ...","[Perez Ring Brewery, Alliance, Corporate, Civi...","[LTT 13904 Jet Universal Int, Alliance, Corpor...","[LTT 13904 Independents, Independent, Democrac...","[LTT 13904 Limited, Alliance, Corporate, None,...","[Allied LTT 13904 Constitution Party, Independ...",NaN,NaN,NaN,NaN,NaN
Bagalis / 2,"[The Misfits of the Galaxy, Independent, Democ...","[MCC 858 Organisation, Federation, Corporate, ...","[Labour of Juan, Federation, Democracy, None, ...",[Applied Starport Safety Technologies Corporat...,"[Juan Industries, Independent, Corporate, None...","[Conservatives of Juan, Independent, Dictators...","[Order of Juan, Independent, Dictatorship, Non...","[Autocracy of LTT 16764, Independent, Dictator...","[United Ross 310 Progressive Party, Federation...","[Bumbur Purple Central Limited, Federation, Co...","[LTT 16764 Nationalists, Independent, Dictator...","[LTT 16764 Progressive Party, Federation, Demo...","[Silver Vision Organisation, Independent, Corp...","[Purple Netcoms Industries, Independent, Corpo...",NaN,NaN,NaN
Belach / 2,"[Brazilian Armada X, Independent, Democracy, N...","[Lem Future Tech, Independent, Corporate, None...","[Galileo Corporation, Independent, Democracy, ...","[Hedeinichs Gold Energy & Co, Federation, Corp...","[Official Hedeinichs Front, Independent, Dicta...","[United Hedeinichs Left Party, Independent, Co...","[DEFENSIVE TRADERS ALLIANCE, Empire, Patronage...","[Movement for Thethys Liberals, Federation, De...","[Defence Force of Thethys, Independent, Dictat...","[Thethys Crimson Transport Ex, Independent, Co...","[Thethys Blue Major & Co, Federation, Corporat...",NaN,NaN,NaN,NaN,NaN,NaN
Cariangan / 2,"[HR 692 Natural Industry, Independent, Corpora...","[Tetekhe Inc, Federation, Corporate, War, HR 692]","[The Ve

#### Counting unique factions

In [16]:
dfa_parsed = df_parsed.count(axis=1).sort_values(ascending=False).reset_index()
dfa_parsed

,index,0
0,Na Chem / 3,17
1,HIP 108701 / 3,16
2,Yavapai / 2,14
3,Wolf 186 / 2,14
4,Bagalis / 2,14
5,LTT 2042 / 2,14
6,Anana / 3,14
7,Findja / 2,13
8,Millese / 2,13
9,HIP 13841 / 2,13


#### Filtering: greater than (gt)

In [17]:
dfb_parsed = dfa_parsed[dfa_parsed[0].ge(13)].copy()
dfl = dfb_parsed.rename(columns={'index': 'Target/Sources', 0: 'Unique Factions'})
dfl

,Target/Sources,Unique Factions
0,Na Chem / 3,17
1,HIP 108701 / 3,16
2,Yavapai / 2,14
3,Wolf 186 / 2,14
4,Bagalis / 2,14
5,LTT 2042 / 2,14
6,Anana / 3,14
7,Findja / 2,13
8,Millese / 2,13
9,HIP 13841 / 2,13


#### Getting URL from the filtered systems

In [20]:
dfr = dfc.loc[:, ['Target/Sources', 'URL']].reset_index(drop=True)
dfr

,Target/Sources,URL
0,Findja / 2,https://edtools.cc/pve?s=Findja&a=rt
1,Findja / 2,https://edtools.cc/pve?s=Findja&a=rt
2,HIP 13841 / 2,https://edtools.cc/pve?s=HIP+13841&a=rt
3,Slatas / 2,https://edtools.cc/pve?s=Slatas&a=rt
4,Chimechilo / 2,https://edtools.cc/pve?s=Chimechilo&a=rt
...,...,...
66,Obates / 2,https://edtools.cc/pve?s=Obates&a=rt
67,HIP 82042 / 2,https://edtools.cc/pve?s=HIP+82042&a=rt
68,BD+37 2416 / 2,https://edtools.cc/pve?s=BD%2B37+2416&a=rt
69,HIP 82042 / 2,https://edtools.cc/pve?s=HIP+82042&a=rt


In [21]:
df_final = pd.merge(dfl, dfr, on = "Target/Sources", how = "left")
df_final = df_final.drop_duplicates().reset_index(drop=True)
df_final

,Target/Sources,Unique Factions,URL
0,Na Chem / 3,17,https://edtools.cc/pve?s=Na+Chem&a=rt
1,HIP 108701 / 3,16,https://edtools.cc/pve?s=HIP+108701&a=rt
2,Yavapai / 2,14,https://edtools.cc/pve?s=Yavapai&a=rt
3,Wolf 186 / 2,14,https://edtools.cc/pve?s=Wolf+186&a=rt
4,Bagalis / 2,14,https://edtools.cc/pve?s=Bagalis&a=rt
5,LTT 2042 / 2,14,https://edtools.cc/pve?s=LTT+2042&a=rt
6,Anana / 3,14,https://edtools.cc/pve?s=Anana&a=rt
7,Findja / 2,13,https://edtools.cc/pve?s=Findja&a=rt
8,Millese / 2,13,https://edtools.cc/pve?s=Millese&a=rt
9,HIP 13841 / 2,13,https://edtools.cc/pve?s=HIP+13841&a=rt


In [22]:
df_final = df_final.loc[df_final['Target/Sources'].str[-1] != '3'].reset_index(drop=True)
df_final

,Target/Sources,Unique Factions,URL
0,Yavapai / 2,14,https://edtools.cc/pve?s=Yavapai&a=rt
1,Wolf 186 / 2,14,https://edtools.cc/pve?s=Wolf+186&a=rt
2,Bagalis / 2,14,https://edtools.cc/pve?s=Bagalis&a=rt
3,LTT 2042 / 2,14,https://edtools.cc/pve?s=LTT+2042&a=rt
4,Findja / 2,13,https://edtools.cc/pve?s=Findja&a=rt
5,Millese / 2,13,https://edtools.cc/pve?s=Millese&a=rt
6,HIP 13841 / 2,13,https://edtools.cc/pve?s=HIP+13841&a=rt
7,Cariangan / 2,13,https://edtools.cc/pve?s=Cariangan&a=rt
8,Misir / 2,13,https://edtools.cc/pve?s=Misir&a=rt


#### Saving final data with candidates

In [23]:
df_final.to_excel('candidates_excel.xlsx')
df_final.to_csv('candidates_csv.csv', sep=';')

#### Checking faction state with inara

In [24]:
df_final = pd.read_csv('candidates_csv.csv', sep=';', index_col=0)
df_final

,Target/Sources,Unique Factions,URL
0,Yavapai / 2,14,https://edtools.cc/pve?s=Yavapai&a=rt
1,Wolf 186 / 2,14,https://edtools.cc/pve?s=Wolf+186&a=rt
2,Bagalis / 2,14,https://edtools.cc/pve?s=Bagalis&a=rt
3,LTT 2042 / 2,14,https://edtools.cc/pve?s=LTT+2042&a=rt
4,Findja / 2,13,https://edtools.cc/pve?s=Findja&a=rt
5,Millese / 2,13,https://edtools.cc/pve?s=Millese&a=rt
6,HIP 13841 / 2,13,https://edtools.cc/pve?s=HIP+13841&a=rt
7,Cariangan / 2,13,https://edtools.cc/pve?s=Cariangan&a=rt
8,Misir / 2,13,https://edtools.cc/pve?s=Misir&a=rt


In [25]:
df_final['INARA'] = None

In [26]:
def check_faction_state(state):
    bag_of_words = ['war', 'election', 'civil war']
    if any(word in state.lower() for word in bag_of_words):
        return True
        

count = 0
for df_index, system_data in enumerate(df_final['Target/Sources']):
    count += 1
    systems = set()
    for key, value in parsed[system_data].items():
        systems.add(value[-1].split(',')[0].strip())
        
    for system in systems:
        param = dict()
        param['search'] = system
        url = 'https://inara.cz/starsystem'
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.71",
        }
        
        # Request
        r = requests.get(url, params=param, headers=headers)

        # Soup
        soup = bs(r.content, 'lxml')

        table_data = soup.body.find('table', class_='tablesorter')

        keep_going = True
        for index, tr in enumerate(table_data.find_all('tr')):
            if not keep_going:
                break
            elif index > 0:
                for i, td in enumerate(tr.find_all('td')):
                    if (i == 3 or i == 4) and check_faction_state(td.text.strip()):
                        df_final.loc[df_index, 'INARA'] = 'war, election or civil war detected'
                        keep_going = False
                        break
    
        sleep(r.elapsed.total_seconds())
        
    if count % 10 == 0:
        print(f'{count} iterations.')

In [27]:
df_final

,Target/Sources,Unique Factions,URL,INARA
0,Yavapai / 2,14,https://edtools.cc/pve?s=Yavapai&a=rt,"war, election or civil war detected"
1,Wolf 186 / 2,14,https://edtools.cc/pve?s=Wolf+186&a=rt,"war, election or civil war detected"
2,Bagalis / 2,14,https://edtools.cc/pve?s=Bagalis&a=rt,None
3,LTT 2042 / 2,14,https://edtools.cc/pve?s=LTT+2042&a=rt,"war, election or civil war detected"
4,Findja / 2,13,https://edtools.cc/pve?s=Findja&a=rt,"war, election or civil war detected"
5,Millese / 2,13,https://edtools.cc/pve?s=Millese&a=rt,"war, election or civil war detected"
6,HIP 13841 / 2,13,https://edtools.cc/pve?s=HIP+13841&a=rt,"war, election or civil war detected"
7,Cariangan / 2,13,https://edtools.cc/pve?s=Cariangan&a=rt,"war, election or civil war detected"
8,Misir / 2,13,https://edtools.cc/pve?s=Misir&a=rt,None


In [28]:
df_final.to_excel('candidates_excel.xlsx')
df_final.to_csv('candidates_csv.csv', sep=';')